In [4]:
# reference: https://github.com/ellisdg/3DUnetCNN/
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["k_fold"] = 5
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["batch_size"] = 1
config["input_shape"] = (1, None, None, None)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
# config["patch_shape"] = (64, 64, 64)  # switch to None to train on the whole image
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 1

In [6]:
from model.load_data_old import *
from model.model import *

d = Data()
d.load_data()
# display current shape of MRI images
for i in d.data:
    print("shape of data: {0}".format(i))
    
# prepare data for training
train_num, valid_num = d.prekfold(config["k_fold"], config["batch_size"])
d.preprocess(batch_size=config["batch_size"], pool_size=config["pool_size"], depth=config["depth"])

Using TensorFlow backend.


shape of data: (144, 512, 512)
shape of data: (261, 336, 336)
shape of data: (128, 224, 256)


In [7]:
result = []
target = []
image = []

def train(config, data, train_num, valid_num):
#     models = []

    print(train_num, valid_num)
    for i in range(data.kfold):
        print ('-'*100)
        print ("Fold:", i)
        model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])
        
        # print(model.summary())
        
        callbacks = get_callbacks(config["weights_file"],
                                initial_learning_rate=config["initial_learning_rate"],
                                learning_rate_drop=config["learning_rate_drop"],
                                learning_rate_patience=config["patience"],
                                early_stopping_patience=config["early_stop"])

        model.fit_generator(generator=data.train_generator(i, batch_size=config["batch_size"]),
                            steps_per_epoch=train_num,
                            epochs=config["n_epochs"],
                            validation_data=data.valid_generator(i),
                            validation_steps=valid_num,
                            callbacks=callbacks,
                            verbose = 1)
        
        for j in d.valid_index:
            valid = d.data[j][i]
            image.append(valid[i])
            target.append(valid[i])
            result.append(model.predict(np.array([valid[i]])))
            
        break

        # models.append(model)

train(config, d, train_num, valid_num)

12 3
----------------------------------------------------------------------------------------------------
Fold: 0
Epoch 1/1
12/12 [==============================] - 7s 597ms/step - loss: 0.2368 - dice_coefficient: 0.7632 - val_loss: 0.2009 - val_dice_coefficient: 0.7991


In [8]:
import ipywidgets as widgets
from ipywidgets import interact, interactive

def show_image(images):
    def show_frame(id):
        length = len(images)
        for i in range(length):
            plt.subplot(1, length, i+1)
            plt.imshow(images[i][0, 0, id, :, :], cmap='gray')
    interact(show_frame, id=widgets.IntSlider(min=0, max=images[0].shape[2]-1, step=1, value=images[0].shape[2]/2))

sel = next(iter(d.valid_index))
show_image([result[0], image[0][None, :, :, :, :], target[0][None, :, :, :, :]])


interactive(children=(IntSlider(value=16, description='id', max=31), Output()), _dom_classes=('widget-interact…

In [9]:
dice = 0
for i in range(len(result)):
    dice += dice_coefficient(target[i], result[i])

print("dice:", dice / len(result))
print("loss:", 1 - dice / len(result)) 

print(dice_coefficient(target, result))
print(np.array(result).shape)

dice: 0.0029124265500517165
loss: 0.9970875734499483
0.0007326275284279208
(3, 1, 1, 32, 32, 32)


In [10]:

# model = unet_model_3d(input_shape=config["input_shape"],
#                               pool_size=config["pool_size"],
#                               initial_learning_rate=config["initial_learning_rate"],
#                               deconvolution=config["deconvolution"],
#                               depth=config["depth"],
#                               n_base_filters=config["n_base_filters"])

# model.load_weights(os.getcwd() + '/model/weight/weights-10--1.00.hdf5.h5')

# for j in d.valid_index:
#     valid = d.data[j][0] # fold = 0
#     shape = valid[0].shape
# #             x = shape[0] // 2
# #             y = shape[1] // 2
# #             z = shape[2] // 2
# #             input = np.array([np.expand_dims(valid[0][x : x + patch_size, y : y + patch_size, z : z + patch_size], axis=0)])
# #             result.append(model.predict(input))
# #             image.append(input)
# #             target.append([np.expand_dims(valid[1][x : x + patch_size, y : y + patch_size, z : z + patch_size], axis=0)])

#     input = np.array([np.expand_dims(ndimage.zoom(valid[0], (32/shape[0], 32/shape[1], 32/shape[2])), axis=0)])
#     target.append(np.expand_dims(ndimage.zoom(valid[1], (32/shape[0], 32/shape[1], 32/shape[2])), axis=0))
#     image.append(input)
#     result.append(model.predict(input))
